## Deliverable 2. Create a Customer Travel Destinations Map.

In [12]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [13]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Georgetown,MY,5.4112,100.3354,78.80,88,20,5.75,few clouds,0.00,0.0
1,1,Thinadhoo,MV,0.5333,72.9333,81.43,76,59,13.96,broken clouds,0.00,0.0
2,2,Chokurdakh,RU,70.6333,147.9167,-5.24,91,100,9.40,overcast clouds,0.00,0.0
3,3,Rikitea,PF,-23.1203,-134.9692,78.57,67,55,7.11,broken clouds,0.00,0.0
4,4,Vila Franca Do Campo,PT,37.7167,-25.4333,61.00,88,40,17.27,light rain,0.14,0.0


In [15]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the min temp you would like for your vacation? "))
max_temp = float(input("What is the max temp you would like for your vacation? "))

What is the min temp you would like for your vacation? 60
What is the max temp you would like for your vacation? 80


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_city_df = city_data_df.loc[(city_data_df['Max Temp']<=max_temp)&( city_data_df['Max Temp']>=70)]
clean_city_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Georgetown,MY,5.4112,100.3354,78.80,88,20,5.75,few clouds,0.0,0.0
3,3,Rikitea,PF,-23.1203,-134.9692,78.57,67,55,7.11,broken clouds,0.0,0.0
9,9,Kapaa,US,22.0752,-159.3190,77.00,83,1,4.38,clear sky,0.0,0.0
10,10,Saint-Philippe,RE,-21.3585,55.7679,78.80,94,20,4.61,few clouds,0.0,0.0
19,19,Ambovombe,MG,-25.1667,46.0833,77.04,91,61,8.52,broken clouds,0.0,0.0


In [32]:
# 4a. Determine if there are any empty rows.
clean_city_df.count()
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = clean_city_df.dropna()
clean_travel_cities.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Georgetown,MY,5.4112,100.3354,78.80,88,20,5.75,few clouds,0.0,0.0
3,3,Rikitea,PF,-23.1203,-134.9692,78.57,67,55,7.11,broken clouds,0.0,0.0
9,9,Kapaa,US,22.0752,-159.3190,77.00,83,1,4.38,clear sky,0.0,0.0
10,10,Saint-Philippe,RE,-21.3585,55.7679,78.80,94,20,4.61,few clouds,0.0,0.0
19,19,Ambovombe,MG,-25.1667,46.0833,77.04,91,61,8.52,broken clouds,0.0,0.0


In [33]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_city_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Georgetown,MY,78.80,few clouds,5.4112,100.3354,
3,Rikitea,PF,78.57,broken clouds,-23.1203,-134.9692,
9,Kapaa,US,77.00,clear sky,22.0752,-159.3190,
10,Saint-Philippe,RE,78.80,few clouds,-21.3585,55.7679,
19,Ambovombe,MG,77.04,broken clouds,-25.1667,46.0833,
21,Moengo,SR,78.42,broken clouds,5.6167,-54.4000,
24,Namibe,AO,75.07,clear sky,-15.1961,12.1522,
28,Chuy,UY,78.39,overcast clouds,-33.6971,-53.4616,
30,Lima,PE,73.40,clear sky,-12.0432,-77.0282,
32,Port Elizabeth,ZA,71.60,scattered clouds,-33.9180,25.5701,


In [34]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [36]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df['Hotel Name']!= ''].copy()
clean_hotel_df.count()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [26]:
# 8a. Create the output File (CSV)
output_data_file = "../Data/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [37]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30,31),zoom_level=1.5)
fig.add_layer(marker_layer)
fig 

Figure(layout=FigureLayout(height='420px'))